In [11]:
from aif360.sklearn.datasets import fetch_bank, fetch_compas, fetch_adult, fetch_german
import pandas as pd
from tqdm.autonotebook import tqdm as tqdm
import random
import numpy as np

In [12]:
#X,y,_ = fetch_adult(subset='train', dropna=True)
#X_test,y_test,_ = fetch_adult(subset='test', dropna=True)
X,y = fetch_german(binary_age=True, dropna=True)
#X,y = fetch_bank(dropna=False)

In [13]:
X

,,,,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,other_parties,residence_since,...,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,sex,marital_status
,sex,age,foreign_worker,,,,,,,,,,,,,,,,,,,,,
0,male,aged,yes,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,none,4.0,...,67.0,none,own,2.0,skilled,1.0,yes,yes,male,single
1,female,young,yes,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,none,2.0,...,22.0,none,own,1.0,skilled,1.0,none,yes,female,div/dep/mar
2,male,aged,yes,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,none,3.0,...,49.0,none,own,1.0,unskilled resident,2.0,none,yes,male,single
3,male,aged,yes,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,guarantor,4.0,...,45.0,none,for free,1.0,skilled,2.0,none,yes,male,single
4,male,aged,yes,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,none,4.0,...,53.0,none,for free,2.0,skilled,2.0,none,yes,male,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,female,aged,yes,no checking,12.0,existing paid,furniture/equipment,1736.0,<100,4<=X<7,3.0,none,4.0,...,31.0,none,own,1.0,unskilled resident,1.0,none,yes,female,div/dep/mar
996,male,aged,yes,<0,30.0,existing paid,used car,3857.0,<100,1<=X<4,4.0,none,4.0,...,40.0,none,own,1.0,high qualif/self emp/mgmt,1.0,yes,yes,male,div/sep
997,male,aged,yes,no checking,12.0,existing paid,radio/tv,804.0,<100,>=7,4.0,none,4.0,...,38.0,none,own,1.0,skilled,1.0,none,yes,male,single


In [14]:
# Adult Pre-Proc
# X.drop(['education-num'], inplace=True, axis=1)
# X_test.drop(['education-num'], inplace=True, axis=1)
# X['income'] = y
# X_test['income'] = y_test

# Credit Pre-Proc
X['credit'] = y

# # Bank Pre-Proc
# X['deposit'] = y
# X.drop(['poutcome', 'contact'], inplace=True, axis=1)
# X.dropna(inplace=True)

In [16]:
# # Adult
# stats = {}
# stats['00'] = []
# stats['01'] = []
# stats['10'] = []
# stats['11'] = []
# for i in X.index:
#     if X.loc[i]['sex'] == 'Female' and X.loc[i]['income'] == '<=50K':
#         stats['01'].append(i)
#     elif X.loc[i]['sex'] == 'Female' and X.loc[i]['income'] == '>50K':
#         stats['00'].append(i)
#     elif X.loc[i]['sex'] == 'Male' and X.loc[i]['income'] == '<=50K':
#         stats['11'].append(i)
#     elif X.loc[i]['sex'] == 'Male' and X.loc[i]['income'] == '>50K':
#         stats['10'].append(i)
# for i in stats:
#     print(i, len(stats[i]))

# stats_test = {}
# stats_test['00'] = []
# stats_test['01'] = []
# stats_test['10'] = []
# stats_test['11'] = []
# for i in X_test.index:
#     if X_test.loc[i]['sex'] == 'Female' and X_test.loc[i]['income'] == '<=50K':
#         stats_test['01'].append(i)
#     elif X_test.loc[i]['sex'] == 'Female' and X_test.loc[i]['income'] == '>50K':
#         stats_test['00'].append(i)
#     elif X_test.loc[i]['sex'] == 'Male' and X_test.loc[i]['income'] == '<=50K':
#         stats_test['11'].append(i)
#     elif X_test.loc[i]['sex'] == 'Male' and X_test.loc[i]['income'] == '>50K':
#         stats_test['10'].append(i)
# for i in stats_test:
#     print(i, len(stats_test[i]))


# Credit
stats = {}
stats['00'] = []
stats['01'] = []
stats['10'] = []
stats['11'] = []
sensitives = {
    'sex':['female', 'male'],
    'foreign_worker': ['yes', 'no'],
    'age': ['young', 'aged']
}
attr = 'sex'
for i in X.index:
    if attr != 'age':
        if X.loc[i][attr] == sensitives[attr][0] and X.loc[i]['credit'] == 'good':
            stats['01'].append(i)
        elif X.loc[i][attr] == sensitives[attr][0] and X.loc[i]['credit'] == 'bad':
            stats['00'].append(i)
        elif X.loc[i][attr] == sensitives[attr][1] and X.loc[i]['credit'] == 'good':
            stats['11'].append(i)
        elif X.loc[i][attr] == sensitives[attr][1] and X.loc[i]['credit'] == 'bad':
            stats['10'].append(i)
    else:
        if i[2] == sensitives[attr][0] and X.loc[i]['credit'] == 'good':
            stats['01'].append(i)
        elif i[2] == sensitives[attr][0] and X.loc[i]['credit'] == 'bad':
            stats['00'].append(i)
        elif i[2] == sensitives[attr][1] and X.loc[i]['credit'] == 'good':
            stats['11'].append(i)
        elif i[2] == sensitives[attr][1] and X.loc[i]['credit'] == 'bad':
             stats['10'].append(i)

# # Bank
# stats = {}
# stats['00'] = []
# stats['01'] = []
# stats['10'] = []
# stats['11'] = []
# for i in X.index:
#     if X.loc[i]['age'] <= 25 and X.loc[i]['income'] == '<=50K':
#         stats['01'].append(i)
#     elif X.loc[i]['age'] == 'Female' and X.loc[i]['income'] == '>50K':
#         stats['00'].append(i)
#     elif X.loc[i]['age'] == 'Male' and X.loc[i]['income'] == '<=50K':
#         stats['11'].append(i)
#     elif X.loc[i]['age'] == 'Male' and X.loc[i]['income'] == '>50K':
#         stats['10'].append(i)
# for i in stats:
#     print(i, len(stats[i]))

In [17]:
for i in stats:
    print(i, len(stats[i]))

00 109
01 201
10 191
11 499


In [19]:
# Adult
# X.to_csv('/media/data_dump/Mohit/facct23_samplebias_data/adult/raw/original_train.csv', index=False)
# X_test.to_csv('/media/data_dump/Mohit/facct23_samplebias_data/adult/raw/original_test.csv', index=False)

# Credit
X.to_csv('/media/data_dump/Mohit/facct23_samplebias_data/run_1/credit/raw/original_full.csv', index=False)

In [20]:
trainIndices = []
testIndices = []
for i in stats:
    random.shuffle(stats[i])
    trainIndices.extend(stats[i][:int(len(stats[i])*0.8)])
    testIndices.extend(stats[i][int(len(stats[i])*0.8):])

In [21]:
X.loc[trainIndices].to_csv('/media/data_dump/Mohit/facct23_samplebias_data/run_1/credit/raw/original_train.csv', index=False)
X.loc[testIndices].to_csv('/media/data_dump/Mohit/facct23_samplebias_data/run_1/credit/raw/original_test.csv', index=False)

In [22]:
def createImbalancedDataframe(csv_path, balance=False, imbalanceFactor=1.0, dataset='adult', betaSplits=False, beta_neg_factor=None, labelFlip=None):
    data = pd.read_csv(csv_path)
    xVals = list(data.index)
    subgroups = {'00':[], '01':[], '10':[], '11':[]}
    for i in xVals:
        if dataset == 'adult':
            # First index (0) -> Female Gender, Second index(0) -> <=50K, (1) -> >50K
            if data.loc[i]['sex'] == 'Female' and data.loc[i]['income'] == '<=50K':
                subgroups['00'].append(i)
            elif data.loc[i]['sex'] == 'Female' and data.loc[i]['income'] == '>50K':
                subgroups['01'].append(i)
            elif data.loc[i]['sex'] == 'Male' and data.loc[i]['income'] == '<=50K':
                subgroups['10'].append(i)
            elif data.loc[i]['sex'] == 'Male' and data.loc[i]['income'] == '>50K':
                subgroups['11'].append(i)
        elif dataset == 'bank':
            # <= 25 -> 0, yes -> 1
            if data.loc[i]['age'] <= 25 and data.loc[i]['deposit'] == 'no':
                subgroups['00'].append(i)
            elif data.loc[i]['age'] <= 25 and data.loc[i]['deposit'] == 'yes':
                subgroups['01'].append(i)
            elif data.loc[i]['age'] > 25 and data.loc[i]['deposit'] == 'no':
                subgroups['10'].append(i)
            elif data.loc[i]['age'] > 25 and data.loc[i]['deposit'] == 'yes':
                subgroups['11'].append(i)
        elif dataset == 'compas':
            if data.loc[i]['race'] == 'African-American' and data.loc[i]['category'] == 'Recidivated':
                subgroups['01'].append(i)
            elif data.loc[i]['race'] == 'African-American' and data.loc[i]['category'] == 'Survived':
                subgroups['00'].append(i)
            elif data.loc[i]['race'] == 'Caucasian' and data.loc[i]['category'] == 'Recidivated':
                subgroups['11'].append(i)
            elif data.loc[i]['race'] == 'Caucasian' and data.loc[i]['category'] == 'Survived':
                subgroups['10'].append(i)
        elif dataset == 'credit':
            if data.loc[i]['sex'] == 'female' and data.loc[i]['credit'] == 'good':
                subgroups['01'].append(i)
            elif data.loc[i]['sex'] == 'female' and data.loc[i]['credit'] == 'bad':
                subgroups['00'].append(i)
            elif data.loc[i]['sex'] == 'male' and data.loc[i]['credit'] == 'good':
                subgroups['11'].append(i)
            elif data.loc[i]['sex'] == 'male' and data.loc[i]['credit'] == 'bad':
                subgroups['10'].append(i)
        elif dataset == 'synthetic':
            if data.loc[i]['sensitive'] == 0 and data.loc[i]['label'] == 1:
                subgroups['01'].append(i)
            elif data.loc[i]['sensitive'] == 0 and data.loc[i]['label'] == 0:
                subgroups['00'].append(i)
            elif data.loc[i]['sensitive'] == 1 and data.loc[i]['label'] == 1:
                subgroups['11'].append(i)
            elif data.loc[i]['sensitive'] == 1 and data.loc[i]['label'] == 0:
                subgroups['10'].append(i)
    if balance == True:
        num_points = min(len(subgroups['00']), len(subgroups['01']), len(subgroups['10']), len(subgroups['11']))
        for i in subgroups:
            subgroups[i] = random.sample(subgroups[i], num_points)
    if dataset == 'adult':
        imbalanceGroup = '01'
        beta_neg_group = '00'
        labelFlipIndex = '01'
        label = 'income'
        flipped_label = '<=50K'
    elif dataset == 'bank':
        imbalanceGroup = '01'
        beta_neg_group = '00'
        labelFlipIndex = '01'
        label = 'deposit'
        flipped_label = 'no'
    elif dataset == 'compas':
        imbalanceGroup = '11'
        beta_neg_group = '10'
        labelFlipIndex = '11'
        label = 'category'
        flipped_label = 'Survived'
    elif dataset == 'credit':
        imbalanceGroup = '01'
        beta_neg_group = '00'
        labelFlipIndex = '01'
        label = 'credit'
        flipped_label = 'bad'
    elif dataset == 'synthetic':
        imbalanceGroup = '01'
        beta_neg_group = '00'
        labelFlipIndex = '01'
        label = 'label'
        flipped_label = 0
    if labelFlip is not None:
        noisyIndices = random.sample(subgroups[labelFlipIndex], int(len(subgroups[labelFlipIndex])*labelFlip))
        for i in noisyIndices:
            data.at[i,label]  = flipped_label
        return data
    else:
        if betaSplits == True:
            subgroups[imbalanceGroup] = random.sample(subgroups[imbalanceGroup], int(len(subgroups[imbalanceGroup])*imbalanceFactor))
            subgroups[beta_neg_group] = random.sample(subgroups[beta_neg_group], int(len(subgroups[beta_neg_group])*beta_neg_factor))
        newDataset = subgroups['00'] + subgroups['01'] + subgroups['10'] + subgroups['11']
        return data.loc[newDataset]

In [23]:
dataset = 'credit'
for beta_pos in tqdm(np.arange(1,11)):
    beta_pos = beta_pos/10
    for beta_neg in np.arange(1,11):
        beta_neg = beta_neg/10
        # Create train set
        temp = createImbalancedDataframe(f'/media/data_dump/Mohit/facct23_samplebias_data/run_1/{dataset}/raw/original_train.csv', imbalanceFactor=beta_pos, dataset=dataset, betaSplits=True, beta_neg_factor=beta_neg)
        temp.to_csv(f'/media/data_dump/Mohit/facct23_samplebias_data/run_1/{dataset}/raw/betaDatasets/imbalance_{beta_pos}_{beta_neg}', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

In [24]:
dataset = 'credit'
for nu in tqdm(np.arange(0,10)):
    nu = nu/10
    temp = createImbalancedDataframe(f'/media/data_dump/Mohit/facct23_samplebias_data/run_1/{dataset}/raw/original_train.csv', balance=False, dataset=dataset, labelFlip=nu)
    temp.to_csv(f'/media/data_dump/Mohit/facct23_samplebias_data/run_1/{dataset}/raw/labelBiasDatasets/imbalance_{nu}', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]